In [1]:
from charm.toolbox.pairinggroup import PairingGroup, G1,G2,GT, ZR, pair as e,extract_key
from hashlib import sha256, md5
from util1 import obj2bytes, read_word_as_bytes, bxor
import numpy as np  
import logging
import pickle
import socketserver
import sys
import import_ipynb
from Hash_Util import *
import copy



class serverReqHandlerV2(socketserver.BaseRequestHandler):
    def __init__(self, request, addr, server):
        super().__init__(request, addr, server)
#接受发送的请求，根据第一项判断请求内容
    def handle(self):
        resp_tup = pickle.loads(self.request.recv(4096))
        if(resp_tup[0] == 0):  # for setup
            self.server.Setup(resp_tup[1])
            data = (1,)
            logging.debug("setup completed")
        elif(resp_tup[0] == 1):
            self.server.Update(resp_tup[1])
            data = (1,)
            logging.debug("update completed")
        elif(resp_tup[0] == 2):
            data = self.server.Search(resp_tup[1])
            logging.debug("search completed")
        elif(resp_tup[0] == 3):
            self.server.updateCTList(resp_tup[1])
            data = (1,)
            logging.debug("Update CT_List Completed")
        elif(resp_tup[0] == 4):
            data = self.server.SendCTList()
            print("服务端的数据")
            print(data)
            logging.debug("SendCTList Completed")
        elif(resp_tup[0] == 5):
            data = self.server.SendRstByR(resp_tup[1])
            logging.debug("Send Rst Completed")
        elif(resp_tup[0] == 6):
            self.server.UpdateCTByRL(resp_tup[1])
            data = (1,)
            logging.debug("CT Update By RL Completed")

        self.request.sendall(pickle.dumps(data))
        logging.debug('handled')
class ODXTServerV2(socketserver.TCPServer):
    def __init__(self, addr, handler_class=serverReqHandlerV2) -> None:
        #初始化参数集合
        self.group = PairingGroup('MNT159')
        self.CT_List = []
        self.R_EDB = {}
        self.R_st = {}
        self.p = -1
        super().__init__(addr, handler_class)

    def Setup(self, res):
        #在R的map中存储对应内容
        EDB, self.p ,EX_R,st= res
        self.R_EDB[EX_R] = EDB
        self.R_st[EX_R] =  st
        print(EX_R)
        

    def Update(self, avax_tup):
        #更新交叉索引表和关键词索引表
        addr, val, α, xtag, upCnt,EX_R,st_s = avax_tup
        self.R_st[EX_R] = st_s
        TSet, XSet = self.R_EDB[EX_R]
        TSet[addr] = (val, α)
        XSet[xtag] = upCnt
        self.R_EDB[EX_R] = (TSet, XSet)
        print(self.R_EDB)
        print("############################")
        print(self.R_st)

    def Search(self, tknlists):
        #云端数据检索，通过发送的token列表
        stokenlist = tknlists[0]
        xtokenlists = tknlists[1]
        EX_R = tknlists[2]
        TSet, XSet = self.R_EDB[EX_R]
        n = len(stokenlist)
        sEOpList = []
        for j in range(n):
            cnt_i = 1
            cnt_j = 0
            sval, α_beta = TSet[stokenlist[j]]
            α, beta = α_beta
            for xt in xtokenlists[j]:
                xtoken_ij = xt
                xtag_ij = pow(xtoken_ij, α, self.p)
                xtag_ij_p = pow(xtag_ij, beta, self.p)
                if(xtag_ij in XSet):
                    cnt_i += 1
                    if(xtag_ij_p in XSet and XSet[xtag_ij] < XSet[xtag_ij_p]):
                        cnt_j += 1
            sEOpList.append((j, sval, cnt_i, cnt_j))
        return (sEOpList,)
    def updateCTList(self,CT):
        #{'CT':[C_0s,CT_1s,CT_2s,CT_3s,CT_4s]}
        CT_real = CT['CT']
        CT_1 = []
        CT_2 = []
        CT_4 = []
        for i in range(len(CT_real[1])):
            CT_1.append(self.group.deserialize(CT_real[1][i]))
            CT_2.append(self.group.deserialize(CT_real[2][i]))
            CT_4.append((CT_real[4][i]))
        
        CT_3 = self.group.deserialize(CT_real[3])
        C_0 = self.group.deserialize(CT_real[0])
        CT_true = [C_0, CT_1,CT_2,CT_3,CT_4]
        print(CT_true)
        print("下面是CT_List")
        if(not(CT_true in self.CT_List)) :
            self.CT_List.append(CT_true)
            
        print(self.CT_List)
    def SendCTList(self):
        CT_S = []
        print("这个是CTList")
        print(self.CT_List)
        for CT in self.CT_List:
            CT_s = []
            CT_s_0 = self.group.serialize(CT[0])
            CT_1s = []
            CT_2s = []
            CT_4s = []
            for i in range(len(CT[1])):
                CT_1s.append(self.group.serialize(CT[1][i]))
                CT_2s.append(self.group.serialize(CT[2][i]))
                CT_4s.append(CT[4][i])
            CT_s_3 = self.group.serialize(CT[3])
            CT_s = [CT_s_0,CT_1s,CT_2s,CT_s_3,CT_4s]
            CT_S.append(CT_s)
        print(CT_S)         
        return CT_S
    def SendRstByR(self,R):
        data =  self.R_st[R]
        return data
    def UpdateCTByRL(self , p):
        #通过撤销列表对权限密文CT中涉及的属性进行更新
        PK_s = p[0]
        RL_s = p[1]
        g2 =  self.group.deserialize(PK_s[10])
        RL = {}
        for sita in RL_s.keys():
            RL[sita] = self.group.deserialize(RL_s[sita])
        print(RL)
        print(self.CT_List)
        for CT in self.CT_List:
            for v in RL.keys():
                for i in range(len(CT[4])):
                    if(CT[4][i] == v):
                        CT[2][i] = CT[2][i] ** (RL[v])
        
        print(self.CT_List)   
        print("属性撤销修改完成")
                    
                
        
        
        
        
        
        
        
        

HOST = 'localhost'
PORT = 50068
if __name__ == "__main__":
    #HOST = sys.argv[1]
    #PORT = int(sys.argv[2])
    server = ODXTServerV2((HOST, PORT), serverReqHandlerV2)
    print("服务器正在运行")
    server.serve_forever()        


importing Jupyter notebook from Hash_Util.ipynb
服务器正在运行
b'\xd5(\xa9-\xc3\x14\xe8J\x9c\xd4)\x05:nb\xf9\x12h\x8b\xe4\x07$\x0e\xcf\x99\xfb\\KmmH\xb0'
[[[188658798659138812106748484111040179471313638123, 235543074286119343473612490574163261557659127264, 14143353602352067020329608307635850080172891672], [439758780069779337438729713705395979249413112090, 413810506326411743693355571982798229049500778298, 69451970084151349242405932032683928866623379137]], [[[460653346893045383735194309626805528326285559006, 591744819852167997108593250509860739285833902569, 23019930530513211616630274946942942105727637382], [522157816194659538581766277873256052110770366185, 516981659085322806682292145606167934844884250336, 213286756410650987571015629056690336837253713179]], [[473713089132681620866398340117715278338866956686, 252046621846493025432118118212979284353958835560, 508509752021038095630609659219283961805494887764], [127948708027897105102473263887509989234569597258, 20723783619653023797992476776460206345

这个是CTList
[[[[188658798659138812106748484111040179471313638123, 235543074286119343473612490574163261557659127264, 14143353602352067020329608307635850080172891672], [439758780069779337438729713705395979249413112090, 413810506326411743693355571982798229049500778298, 69451970084151349242405932032683928866623379137]], [[[460653346893045383735194309626805528326285559006, 591744819852167997108593250509860739285833902569, 23019930530513211616630274946942942105727637382], [522157816194659538581766277873256052110770366185, 516981659085322806682292145606167934844884250336, 213286756410650987571015629056690336837253713179]], [[473713089132681620866398340117715278338866956686, 252046621846493025432118118212979284353958835560, 508509752021038095630609659219283961805494887764], [127948708027897105102473263887509989234569597258, 20723783619653023797992476776460206345119820738, 389473639387312591192256681235504820044342232035]], [[287887099677627453891121826019755027688524945387, 197435477364887835126

这个是CTList
[[[[188658798659138812106748484111040179471313638123, 235543074286119343473612490574163261557659127264, 14143353602352067020329608307635850080172891672], [439758780069779337438729713705395979249413112090, 413810506326411743693355571982798229049500778298, 69451970084151349242405932032683928866623379137]], [[[460653346893045383735194309626805528326285559006, 591744819852167997108593250509860739285833902569, 23019930530513211616630274946942942105727637382], [522157816194659538581766277873256052110770366185, 516981659085322806682292145606167934844884250336, 213286756410650987571015629056690336837253713179]], [[473713089132681620866398340117715278338866956686, 252046621846493025432118118212979284353958835560, 508509752021038095630609659219283961805494887764], [127948708027897105102473263887509989234569597258, 20723783619653023797992476776460206345119820738, 389473639387312591192256681235504820044342232035]], [[287887099677627453891121826019755027688524945387, 197435477364887835126

KeyboardInterrupt: 